<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 - lab 2: Testing the database

As usual we have to tell Jupyter to allow SQL:

In [148]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


And then we import our movie database

In [149]:
%sql sqlite:///movies.sqlite

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [150]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///movies.sqlite
Done.


[]

## SQL statements to review at the lab session

Write SQL code for the following tasks:

+ Show the names of all movies.

In [151]:
%%sql

SELECT movie_name
FROM movies

 * sqlite:///movies.sqlite
Done.


movie_name
Alien
Pulp Fiction
Interstellar


+ Show the performance dates for one of the movies.

In [152]:
%%sql

SELECT performance_date
FROM performances
JOIN movies
USING (imdb_key)
WHERE movie_name = 'Interstellar' AND production_year = 2014
GROUP BY performance_date

 * sqlite:///movies.sqlite
Done.


performance_date
2023-02-23
2023-02-24


+ Show all data concerning performances at a given theatere
  on a given date.

In [153]:
%%sql

SELECT * 
FROM performances
WHERE theater_name = 'Lund' AND performance_date = '2023-02-23'

 * sqlite:///movies.sqlite
Done.


performance_id,start_time,performance_date,theater_name,imdb_key
1,12:00,2023-02-23,Lund,tt0078748
2,15:00,2023-02-23,Lund,tt0110912
3,18:00,2023-02-23,Lund,tt0816692


+ List all customers

In [154]:
%%sql

SELECT full_name
FROM customers

 * sqlite:///movies.sqlite
Done.


full_name
Filip Lennhager
Jonathan Do
Felix R??d??n


+ List all tickets

In [155]:
%%sql

SELECT *
FROM tickets

 * sqlite:///movies.sqlite
Done.


ticket_id,performance_id,username
56dbb09bcfe46e487afd8fcdda7c1dda,1,fi1234le
63d89a6f6804226b6535e34ce9a3eb77,2,fe1234r??
0047a853e345759a0e11f08b2e95cb40,3,jo1234do


## Things to try out yourselves (not reviewed at the lab session)

During lab 3 you'll write a REST service for the database
you just created, and for the server to work properly, the
things below should work.

If you have any questions about the material below, feel
free to ask them at the QA sessions (see Moodle for how to
sign up for them).


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [156]:
%%sql

SELECT username 
FROM customers

 * sqlite:///movies.sqlite
Done.


username
fe1234r??
fi1234le
jo1234do


In [157]:
%%sql

SELECT performance_id
FROM performances


 * sqlite:///movies.sqlite
Done.


performance_id
1
2
3
4
5
6


In [158]:
%%sql

INSERT
INTO       tickets (performance_id, username)
VALUES  (6,'fi1234le');

 * sqlite:///movies.sqlite
1 rows affected.


[]

In newer versions of SQLite (since version 3.35, released
  in March 2021), and in
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html),
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


which would return the generated `s_id` for the new
  student.

  If your SQLite version is older than 3.35, and you can't
  upgrade, you can instead use the following idea: each row
  in a SQLite3 table has a `rowid` attribute, it is a unique
  integer which essentially tells in which order the rows
  were inserted, and it's not displayed in queries unless we
  ask for it. SQLite3 also have a function,
  `last_insert_rowid()`, which returns the `rowid` of the
  last inserted row of a table, so we can see the `s_id` of
  the most recently inserted student with the following
  query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [159]:
%%sql

SELECT ticket_id
FROM   tickets
WHERE  rowid = last_insert_rowid();

 * sqlite:///movies.sqlite
Done.


ticket_id
89ec1a801faf86cd5b2b410f0e13c555


+ Try to insert two movie theaters with the same name (this
  should fail).

In [160]:
%%sql

INSERT
INTO        theaters
VALUES      ('Stockholm', 100),
            ('Stockholm', 200);


 * sqlite:///movies.sqlite
(sqlite3.IntegrityError) UNIQUE constraint failed: theaters.theater_name
[SQL: INSERT
INTO        theaters
VALUES      ('Stockholm', 100),
            ('Stockholm', 200);]
(Background on this error at: http://sqlalche.me/e/gkpj)


+ Try to insert a performance where the theater doesn't
  exist in the database (this should fail).

In [161]:
%%sql

INSERT
INTO       performances(start_time, performance_date, theater_name, imdb_key)
VALUES     ('19:30', '2023-01-01', 'Kiruna', 'tt0078748' )

 * sqlite:///movies.sqlite
(sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT
INTO       performances(start_time, performance_date, theater_name, imdb_key)
VALUES     ('19:30', '2023-01-01', 'Kiruna', 'tt0078748' )]
(Background on this error at: http://sqlalche.me/e/gkpj)


+ Create a ticket where either the user or the performance
  doesn't exist (this should fail).

In [162]:
%%sql

INSERT
INTO       tickets (performance_id, username)
VALUES     (1, 'finnsEj')

 * sqlite:///movies.sqlite
(sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT
INTO       tickets (performance_id, username)
VALUES     (1, 'finnsEj')]
(Background on this error at: http://sqlalche.me/e/gkpj)


In [163]:
%%sql

INSERT
INTO       tickets (performance_id, username)
VALUES     (99, 'fi1234le')

 * sqlite:///movies.sqlite
(sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT
INTO       tickets (performance_id, username)
VALUES     (99, 'fi1234le')]
(Background on this error at: http://sqlalche.me/e/gkpj)


In [164]:
%%sql

SELECT *
FROM tickets

 * sqlite:///movies.sqlite
Done.


ticket_id,performance_id,username
56dbb09bcfe46e487afd8fcdda7c1dda,1,fi1234le
63d89a6f6804226b6535e34ce9a3eb77,2,fe1234r??
0047a853e345759a0e11f08b2e95cb40,3,jo1234do
89ec1a801faf86cd5b2b410f0e13c555,6,fi1234le
